# GOATHerd Base

In [2]:
import sys

sys.path.append("/xpal-src/goatherd")

from goatherd import *
# Connect to the GOATs
neo4jconfig=get_mgc("gegoat")
print(neo4jconfig)
neo4jdb=Neo4jGoat(neo4jconfig)

{'username': 'neo4j', 'password': 'theansweris42', 'dbname': 'gegoat', 'dburl': 'host.docker.internal:7687', 'dbport': '7687'}


In [3]:
labels=[
    'Person',
    'Brand',
    'Company',
    'NGO',
    'Govt Body',
    'Node',
]
def update_keystones(goat):
    k1=goat.add_node(nodeid="__keystone1")
    k2=goat.add_node(nodeid="__keystone2")
    goat.update_labels("__keystone1",labels)
    goat.update_labels("__keystone2",labels)

    k1.isthesameas.add(k2)
    k1.linkedto.add(k2)
    k2.isthesameas.add(k1)
    k2.linkedto.add(k1)
    goat.repo.save(k2)
    goat.repo.save(k1)

In [4]:
print(neo4jdb.get_compostion())
update_keystones(neo4jdb)
print(neo4jdb.get_compostion())
neo4jdb.get_compostion()

[{'Company': 2}, {'Contract': 2}, {'Person': 2}, {'Funding Source': 2}, {'Brand': 2}, {'Partner': 2}, {'Govt Body': 2}, {'Investment Officer': 8}, {'Grant': 2}, {'NGO': 2}, {'Node': 86}, {'Investment': 80}]
__keystone1
Node exists
__keystone2
Node exists
[{'Company': 2}, {'Contract': 2}, {'Person': 2}, {'Funding Source': 2}, {'Brand': 2}, {'Partner': 2}, {'Govt Body': 2}, {'Investment Officer': 8}, {'Grant': 2}, {'NGO': 2}, {'Node': 86}, {'Investment': 80}]


[{'Company': 2},
 {'Contract': 2},
 {'Person': 2},
 {'Funding Source': 2},
 {'Brand': 2},
 {'Partner': 2},
 {'Govt Body': 2},
 {'Investment Officer': 8},
 {'Grant': 2},
 {'NGO': 2},
 {'Node': 86},
 {'Investment': 80}]

In [11]:
company={"nodeid": "U85300DL2022PTC402649", "companyname": "HCAH SENIOR CARE PRIVATE LIMITED", "urls": ["https://www.tofler.in/hcah-senior-care-private-limited/company/U85300DL2022PTC402649", "https://www.hcah.in"], "labels": ["Company", "Organization", "Node"]}

In [12]:
company


{'nodeid': 'U85300DL2022PTC402649',
 'companyname': 'HCAH SENIOR CARE PRIVATE LIMITED',
 'urls': ['https://www.tofler.in/hcah-senior-care-private-limited/company/U85300DL2022PTC402649',
  'https://www.hcah.in'],
 'labels': ['Company', 'Organization', 'Node']}

In [18]:
n=neo4jdb.add_node(**company)

U85300DL2022PTC402649
Node exists


In [20]:
neo4jdb.update_labels(n.nodeid,company['labels'])

:Company:Node:Organization

In [ ]:
#Get the list of relationships defined in the IncDig excel sheet
file_path = "/xpal-data/IncDig.xlsx"
df = pd.read_excel(file_path,"RelLabels")
print(df)
reldict=df.to_dict(orient="records")
def get_rel_labels(rel):
    return [x for x in reldict if x['rel']==rel]
file_path = "/xpal-data/IncDig.xlsx"
df = pd.read_excel(file_path)
rellist=df.to_dict(orient="records")
rellist,reldict

# Write df back to the same worksheet 
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    df.to_excel(writer, sheet_name="RelLabels",index=False)

for rel in rellist:
    labels=get_rel_labels(rel['story'])
    sourcelabels=list(set(["Node"]+list(neo4jdb.get_labels(rel['source']))+[x.lstrip().rstrip() for x in labels['sourcelabels'].lstrip().strip().split(",")]))
    targetlabels=list(set(["Node"]+list(neo4jdb.get_labels(rel['target']))+[x.lstrip().rstrip() for x in labels['targetlabels'].lstrip().strip().split(",")]))
    neo4jdb.update_labels(rel['source'],sourcelabels)
    neo4jdb.update_labels(rel['target'],targetlabels)


    
    

In [ ]:
for row in sherrymap:
    rels=[]
    invnodeid=row['invid'].lower().lstrip().rstrip().replace("-","")
    source=neo4jdb.repo.match(Node,invnodeid).first()
    if source is None:
        print("Adding Investment")
        add_investment(invdict)
    story=row['story'].lstrip().rstrip()
    if row['qtarget']!="":
        try:
            qtarget=neo4jdb.repo.match(Node,row['qtarget'].lstrip().strip()).first()
            if qtarget is not None:
                print(source.nodeid,story,qtarget.nodeid)
        except:
            print(row)
    starget=neo4jdb.repo.match(Node,row['sectarget'].lstrip().strip()).first()
    if starget is not None:
        print(source.nodeid,story,starget.nodeid)
    